In [ ]:
%pip install stable-baselines3[extra]

In [1]:
from stable_baselines3.common.env_checker import check_env
from games.tileman.envs.solo_player_env import SoloPlayerEnv

env = SoloPlayerEnv()
check_env(env, warn=True)
env.close()

c:\Users\adoma\AppData\Local\Programs\Python\Python39\lib\site-packages\stable_baselines3\common\env_checker.py:55: UserWarning: It seems that your observation  is an image but its `dtype` is (int8) whereas it has to be `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  warnings.warn(
c:\Users\adoma\AppData\Local\Programs\Python\Python39\lib\site-packages\stable_baselines3\common\env_checker.py:63: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
c:\Users\adoma\AppData\Local\Programs\Python\Python39\lib\site-packages\stable_baselines3\common\env_checker.py:76: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom features extractor cf. https://stable-b

In [ ]:
from games.tileman.envs.solo_player_env import SoloPlayerEnv
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env

# Instantiate the env
# Train the agent
model = A2C("MlpPolicy", vec_env, verbose=1)

Using cpu device


In [2]:
import numpy as np
import games.tileman.envs.solo_player_env
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

rng = np.random.default_rng(0)
env = make_vec_env(
    "tileman-solo-v0",
    n_envs=1,
    rng=rng,
    post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],  # for computing rollouts
)

def train_expert():
    expert = PPO(
        policy=MlpPolicy,
        env=env,
        seed=0,
        batch_size=64,
        ent_coef=0.0,
        learning_rate=0.0003,
        n_epochs=10,
        n_steps=64,
    )
    expert.learn(100_000)  # Note: change this to 100_000 to train a decent expert.
    return expert

In [ ]:
def sample_expert_transitions():
    expert = train_expert()

    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=None, min_episodes=50),
        rng=rng,
    )
    return rollout.flatten_trajectories(rollouts)

transitions = sample_expert_transitions()
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [ ]:
evaluation_env = make_vec_env(
    "tileman-solo-v0",
    rng=rng,
    env_make_kwargs={"render_mode": "human"},  # for rendering
)

print("Evaluating the untrained policy.")
reward, _ = evaluate_policy(
    bc_trainer.policy,  # type: ignore[arg-type]
    evaluation_env,
    n_eval_episodes=3,
    render=False,  # comment out to speed up
)
print(f"Reward before training: {reward}")

print("Training a policy using Behavior Cloning")
bc_trainer.train(n_epochs=10)

print("Evaluating the trained policy.")
reward, _ = evaluate_policy(
    bc_trainer.policy,  # type: ignore[arg-type]
    evaluation_env,
    n_eval_episodes=3,
    render=False,  # comment out to speed up
)
print(f"Reward after training: {reward}")

Evaluating the untrained policy.
Reward before training: 0.0
Training a policy using Behavior Cloning


0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00139 |
|    entropy        | 1.39     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 129      |
|    loss           | 1.38     |
|    neglogp        | 1.39     |
|    prob_true_act  | 0.25     |
|    samples_so_far | 32       |
--------------------------------


1batch [00:00, 33.24batch/s]

Evaluating the trained policy.


Reward after training: 0.0


In [ ]:
import pygame
from stable_baselines3.common.env_util import make_vec_env
from games.tileman.envs.solo_player_env import SoloPlayerEnv
from games.tileman.envs.solo_player_env import SoloPlayerEnv
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env

# Instantiate the env
# Train the agent

vec_env = make_vec_env(SoloPlayerEnv, n_envs=1, env_kwargs=dict(grid_size=40, vision_range=5, max_steps=300))
model = PPO("MlpPolicy", vec_env, verbose=1)
obs = vec_env.reset()

def run_eval():
    rewards = 0
    obs = vec_env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = vec_env.step(action)
        vec_env.render()
        
        rewards += reward
        
        if done:
            print(f"Episode finished after {1} timesteps, total rewards: {rewards}")
            rewards = 0
            obs = vec_env.reset()

while True:
    run_eval()

env.close()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
Using cpu device
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 timesteps, total rewards: [0.]
Episode finished after 1 time